# Ergodicity Insurance - Volatility Simulation - Parallel

In [ ]:
!pip install --user --upgrade --force-reinstall git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits

  Cloning https://github.com/AlexFiliakov/Ergodic-Insurance-Limits to /tmp/pip-req-build-zjohftyb
  Running command git clone --filter=blob:none --quiet https://github.com/AlexFiliakov/Ergodic-Insurance-Limits /tmp/pip-req-build-zjohftyb
  Resolved https://github.com/AlexFiliakov/Ergodic-Insurance-Limits to commit d8faba5e4238c624ab1447c920ac3801d8cf59b8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sys
import os
import glob

# Add user site-packages to the system path
user_site_packages = os.path.expanduser("~/.local/lib/python*/site-packages")
sys.path.extend(glob.glob(user_site_packages)) # Use glob to handle potential version differences

import ergodic_insurance as ei

ei.__version__

'0.3.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys # Import sys
import glob # Import glob

# Define the directory containing the module from Google Drive
module_dir = "/content/drive/MyDrive/Colab Notebooks"

# Add the directory to sys.path if it's not already there
if module_dir not in sys.path:
    sys.path.append(module_dir)
    print(f"Added {module_dir} to sys.path within")

# Now import the module from the file in Google Drive
try:
    from run_vol_sim_colab import run_vol_sim
    print("Successfully imported run_vol_sim from run_vol_sim_colab.py")
except ModuleNotFoundError:
    print("Could not import run_vol_sim from run_vol_sim_colab.py within.")
    print("Please ensure that run_vol_sim_colab.py exists directly in the /content/drive directory and contains the 'run_vol_sim' function.")
    # Re-raise the error so the user sees it
    raise
except ImportError as e:
    print(f"An error occurred during import within: {e}")
    # Re-raise the error so the user sees it
    raise

Added /content/drive/MyDrive/Colab Notebooks to sys.path within cell WfwYS382ns8Y
Successfully imported run_basic_simulation from run_basic_simulation_colab.py within cell WfwYS382ns8Y


In [ ]:
# import os
# import glob

# # Get the current working directory
# current_directory = os.getcwd()
# print(f"Current working directory: {current_directory}")

# # List all .pkl files in the current directory
# pkl_files = glob.glob(os.path.join(current_directory, '*.pkl'))

# # Delete each file
# if pkl_files:
#     for file_path in pkl_files:
#         os.remove(file_path)
#         print(f"Deleted: {file_path}")
#     print("All .pkl files in the current working directory have been deleted.")
# else:
#     print("No .pkl files found in the current working directory.")

In [ ]:
from tqdm.notebook import tqdm  # progress bar
from itertools import product
import os
from joblib import Parallel, delayed
import shutil # Import shutil for file operations

# Integrate joblib with tqdm for per-task progress
class TqdmJoblib(tqdm):
    def __enter__(self):
        from joblib import parallel
        self._original = parallel.BatchCompletionCallBack
        outer = self
        class TqdmBatchCompletionCallback(parallel.BatchCompletionCallBack):
            def __call__(self, *args, **kwargs):
                outer.update(n=self.batch_size)
                return super().__call__(*args, **kwargs)
        parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
        return super().__enter__()
    def __exit__(self, exc_type, exc, tb):
        from joblib import parallel
        parallel.BatchCompletionCallBack = self._original
        return super().__exit__(exc_type, exc, tb)

NUM_SIMULATIONS = 100_000
SIM_YEARS = 25
PRICING_SIMULATIONS = 500_000  # For premium estimation

initial_assets_list = [5_000_000, 10_000_000, 25_000_000]
atr_list = [0.8, 0.9, 1.0, 1.1, 1.2]
ebitabl_list = [0.1, 0.125, 0.15]
deductible_list = [0, 50_000, 100_000, 250_000, 500_000]
loss_ratio_list = [0.6, 0.7, 0.8]

total_combos = (
    len(initial_assets_list)
    * len(atr_list)
    * len(ebitabl_list)
    * len(deductible_list)
    * len(loss_ratio_list)
)

# Use all but one CPU core
_cpu = os.cpu_count() or 1
n_jobs = max(_cpu, 1)

print(f'Available cores: {n_jobs}')

# Define the destination directory in your Google Drive
# Make sure 'My Drive/results' exists in your Google Drive
gdrive_results_dir = '/content/drive/My Drive/Colab Notebooks/results'
os.makedirs(gdrive_results_dir, exist_ok=True) # Create the directory if it doesn't exist

# Parallel execution with progress bar
with TqdmJoblib(total=total_combos, desc="Parameter sets", leave=True):
    Parallel(n_jobs=n_jobs, prefer="processes", batch_size=1)(
        delayed(run_vol_sim)(
            index, ia, atr, ebitabl, ded, lr,
            NUM_SIMULATIONS, SIM_YEARS, PRICING_SIMULATIONS,
            filepath=gdrive_results_dir
        )
        for index, (ia, atr, ebitabl, ded, lr) in enumerate(product(
            initial_assets_list,
            atr_list,
            ebitabl_list,
            deductible_list,
            loss_ratio_list
        ))
    )

Available cores: 44


Parameter sets:   0%|          | 0/675 [00:00<?, ?it/s]